<a href="https://colab.research.google.com/github/JonDee2023/PredictingStockVolatility/blob/main/PredictingStockVolatility.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Installed pydantic, pydantic_settings, python-dotenv
!pip install pydantic
!pip install pydantic-settings
!pip install python-dotenv

In [3]:
import pandas as pd
import requests

import argparse

from sqlalchemy import create_engine
from time import time
from dotenv import load_dotenv
import os

In [4]:
from dotenv import load_dotenv
import os

# Attempt to load environment variables from .env file
try:
    load_dotenv("C:/Users/lenovo/myenv.env")
    print("Environment variables loaded successfully.")
except Exception as e:
    print("Error loading environment variables:", e)

# Access environment variables and print them for debugging
print("Alpha API Key:", os.getenv("ALPHA_API_KEY"))
print("DB Name:", os.getenv("DB_NAME"))
print("Model Directory:", os.getenv("MODEL_DIRECTORY"))

Environment variables loaded successfully.
Alpha API Key: None
DB Name: None
Model Directory: None


In [ ]:
# Import settings
from config import settings

In [ ]:
user = "postgres"
password = "root"
host = "localhost"
database = "dsprojects"
port = "5432"

In [ ]:
engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{database}')
engine.connect()

In [ ]:
ticker ='SHEL'
output_size='compact'
table_name = ticker


url = (
        "https://learn-api.wqu.edu/1/data-services/alpha-vantage/query?"
        "function=TIME_SERIES_DAILY&"
        f"symbol={ticker}&"
        f"outputsize={output_size}&"
        f"datatype=json&"
        f"apikey={settings.alpha_api_key}"
        )

print(url)

In [ ]:
response = requests.get(url=url)
print("response code:" response.status_code)
shell_stock_data = response.json['Time Series (Daily)']

In [ ]:
def get_daily(ticker, output_size='full'):

    """Get daily time series of an equity from AlphaVantage API.

    Parameters
    ----------
    ticker : str
        The ticker symbol of the equity.
    output_size : str, optional
        Number of observations to retrieve. "compact" returns the
        latest 100 observations. "full" returns all observations for
        equity. By default "full".

    Returns
    -------
    pd.DataFrame
        Columns are 'open', 'high', 'low', 'close', and 'volume'.
        All are numeric.
    """
    # Create URL (8.1.5)
    url = (
        "https://learn-api.wqu.edu/1/data-services/alpha-vantage/query?"
        "function=TIME_SERIES_DAILY&"
        f"symbol={ticker}&"
        f"outputsize={output_size}&"
        f"datatype=json&"
        f"apikey={settings.alpha_api_key}"
        )


    # Send request to API (8.1.6)
    response = requests.get(url=url)

    # Extract JSON data from response (8.1.10)
    response_data = response.json()


    # Read data into DataFrame (8.1.12 & 8.1.13)
    stock_data = response_data['Time Series (Daily)']
    df = pd.DataFrame.from_dict(stock_data, orient='index', dtype=float)


    # Convert index to `DatetimeIndex` named "date" (8.1.14)
    df.index = pd.to_datetime(df.index)
    df.index.name = 'date'


    # Remove numbering from columns (8.1.15)
    df.columns = [c.split('. ')[1] for c in df.columns]

    # Return DataFrame
    return df

In [ ]:
def insert_table(table_name, records, if_exists='fail'):

    """Insert DataFrame into PostgreSQL database as table

    Parameters
    ----------
    table_name : str
    records : pd.DataFrame
    if_exists : str, optional
        How to behave if the table already exists.

        - 'fail': Raise a ValueError.
        - 'replace': Drop the table before inserting new values.
        - 'append': Insert new values to the existing table.

        Dafault: 'fail'

    Returns
    -------
    dict
        Dictionary has two keys:

        - 'transaction_successful', followed by bool
        - 'records_inserted', followed by int
    """

    n_inserted = records.to_sql(name=table_name, con=engine, if_exists=if_exists)

    return {
            'transaction_successful': True,
            'records_inserted': n_inserted
        }